In [1]:
import numpy as np
import os, sys
sys.path.append('../')
from data_functions import degree2radians, mapping_map_to_sphere, process_df, get_XYZV, create_spherical_heatmap_vals, get_lines
import pandas as pd
df = pd.read_csv("../data/GlobalLandTemperaturesByCity.csv")
df = df[df['AverageTemperature'].notna()]
df['Year'] = np.around([int(dt.split('-')[0]) for dt in df['dt']], -1)
tdf = process_df(df, ['Country'], 'Latitude', 'Longitude')

In [2]:
tdf = df.groupby(['Country', 'Year']).mean().reset_index()
tdf.head()

,Country,Year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,X,Y,Z
0,Afghanistan,1830,13.092375,2.144333,34.882000,69.842000,0.282603,0.769918,0.571762
1,Afghanistan,1840,13.052596,2.031222,34.535942,68.407167,0.302857,0.764872,0.566749
2,Afghanistan,1850,13.225438,1.829760,34.248273,68.243741,0.306108,0.766607,0.562587
3,Afghanistan,1860,13.067468,1.937233,34.302461,68.141473,0.307298,0.765519,0.563362
4,Afghanistan,1870,13.617368,1.932522,34.158750,68.178750,0.307324,0.767058,0.561281


In [3]:
import pycountry
keys = [list(pycountry.countries)[i].alpha_3 for i in range(len(pycountry.countries))]
names = [list(pycountry.countries)[i].name for i in range(len(pycountry.countries))]
conv = { v:k for k,v in zip(keys,names) }
def match_name(x):
    try:
        return conv[x]
    except KeyError as e:
        for  n in names:
            if x in n:
                return conv[n]
        return 'N/A'
tdf['codes'] = tdf.apply(lambda x: match_name(x['Country']), axis=1)

In [4]:
tdf.head()

,Country,Year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,X,Y,Z,codes
0,Afghanistan,1830,13.092375,2.144333,34.882000,69.842000,0.282603,0.769918,0.571762,AFG
1,Afghanistan,1840,13.052596,2.031222,34.535942,68.407167,0.302857,0.764872,0.566749,AFG
2,Afghanistan,1850,13.225438,1.829760,34.248273,68.243741,0.306108,0.766607,0.562587,AFG
3,Afghanistan,1860,13.067468,1.937233,34.302461,68.141473,0.307298,0.765519,0.563362,AFG
4,Afghanistan,1870,13.617368,1.932522,34.158750,68.178750,0.307324,0.767058,0.561281,AFG


In [5]:
import plotly.graph_objects as go

years = tdf['Year'].unique()
years.sort()
years_sliders = []

for year in years:
    ttdf = tdf[tdf['Year'] == year]

    years_sliders.append(go.Choropleth(
        visible=False,
        locations = ttdf['codes'],
        z = ttdf['AverageTemperature'],
        text = ttdf['Country'],
        colorscale = 'RdBu',
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_ticksuffix = '°C',
        colorbar_title = 'Temperature',
        zmin=tdf['AverageTemperature'].min(), zmax=tdf['AverageTemperature'].max(), zmid=0
    ))

fig = go.Figure(data=years_sliders)

steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Year: " + str(years[i])}],
              label=str(years[i])  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=len(years),
    currentvalue={"prefix": "Steps: "},
    pad={"t": 50},
    steps=steps,
)]

geo=dict(
        showframe=False,
        showcoastlines=False,
        #projection_type='orthographic'
    )
fig.update_layout(
    sliders=sliders,
    geo=geo
)
fig.data[0].visible = True
fig.write_html('../plots/complex_globe_flat.html')

fig.show()

In [6]:
ttdf

,Country,Year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,X,Y,Z,codes
18,Afghanistan,2010,15.090014,0.628499,34.158750,68.178750,0.307324,0.767058,0.561281,AFG
46,Albania,2010,16.605304,0.411717,40.990000,19.170000,0.712968,0.247863,0.655927,ALB
73,Algeria,2010,19.071761,0.407687,35.206000,3.518000,0.814424,0.050421,0.576199,DZA
89,Angola,2010,22.302083,0.973621,-12.320000,14.396667,0.945526,0.242676,-0.213250,AGO
108,Argentina,2010,17.585128,0.407253,-32.450625,-63.242188,0.378398,-0.748023,-0.534300,ARG
...,...,...,...,...,...,...,...,...,...,...
3282,Venezuela,2010,26.359036,0.477524,10.087250,-67.280000,0.379870,-0.907219,0.175126,VEN
3301,Vietnam,2010,25.572213,0.363113,14.572667,106.810000,-0.278876,0.923004,0.250677,N/A
3316,Yemen,2010,26.776636,0.674174,13.660000,44.585000,0.691993,0.682040,0.236160,YEM
3333,Zambia,2010,21.367181,0.656451,-13.860000,27.925000,0.857073,0.454434,-0.239390,ZMB
